In [1]:
import pandas as pd

In [2]:
import json
with open ('data/hate_memes_train_merged.json', 'r') as r_file:
    my_file = json.load(r_file)

ids = []
img = []
label = []
text = []

for obj in my_file:
    ids.append(obj['id'])
    img.append(obj['img'])
    label.append(obj['label'])
    text.append(obj['text'])

df = pd.DataFrame({'id': ids, 'img': img,'label': label,'text':text})


    


In [3]:
print(df)

         id            img  label  \
0     42953  img/42953.png      0   
1     23058  img/23058.png      0   
2     13894  img/13894.png      0   
3     37408  img/37408.png      0   
4     82403  img/82403.png      0   
...     ...            ...    ...   
8495  10423  img/10423.png      1   
8496  98203  img/98203.png      1   
8497  36947  img/36947.png      1   
8498  16492  img/16492.png      1   
8499  15937  img/15937.png      1   

                                                   text  
0      its their character not their color that matters  
1     don't be afraid to love again everyone is not ...  
2                              putting bows on your pet  
3     i love everything and everybody! except for sq...  
4     everybody loves chocolate chip cookies, even h...  
...                                                 ...  
8495                  nobody wants to hang auschwitz me  
8496  when god grants you a child after 20 years of ...  
8497  gays on social media: equal

In [4]:
import io
from google.cloud import vision
import os

password_file = "My Project-e17cafef88b1.json" 

def get_all(path, password_file=password_file):
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = password_file
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    
    
    response = client.label_detection(image=image)
    labels = response.label_annotations
    ret_label = [label.description for label in labels]
    print(ret_label)

    
    objects = client.object_localization(
        image=image).localized_object_annotations
    ret_object = [(object_.name, object_.score) for object_ in objects]
    print(ret_object)

    return {'path': path,'labels': ret_label, 'objects': ret_object}

In [5]:
labels = []
objects = []
paths = []
for i in range(8499):
    path = "data/"+str(df['img'][i])
    
    dicti = get_all(path=path)
    labels.append(dicti['labels'])
    objects.append(dicti['objects'])
for i in range(8499):
    
    paths.append(path)
    

['Forehead', 'Chin', 'Jaw', 'Publication', 'Font', 'Adaptation', 'Poster', 'Photo caption', 'Advertising', 'No expression']
[('Person', 0.8108208775520325), ('Person', 0.761947751045227), ('Clothing', 0.6507939696311951)]
['Facial expression', 'Smile', 'Flash photography', 'Happy', 'Sleeve', 'Gesture', 'Tie', 'Suit', 'Interaction', 'Font']
[('Person', 0.8804235458374023), ('Person', 0.861129105091095), ('Tie', 0.8482463955879211), ('Outerwear', 0.6752802729606628), ('Clothing', 0.5447781682014465)]
['Cat', 'White', 'Felidae', 'Carnivore', 'Blue', 'Small to medium-sized cats', 'Whiskers', 'Grey', 'Red', 'Snout']
[('Cat', 0.9049119353294373), ('Bowtie', 0.7293491363525391), ('Cat', 0.727033257484436)]
['Dog', 'Photograph', 'Light', 'Dog breed', 'Carnivore', 'Black', 'Mammal', 'Companion dog', 'Snout', 'Sporting Group']
[('Dog', 0.931591272354126), ('Dog', 0.8330581784248352)]
['Musical instrument', 'Font', 'Music', 'Entertainment', 'Event', 'Photo caption', 'Darkness', 'Suit', 'Formal we

KeyboardInterrupt: 

In [6]:
paths = []
for i in range(len(labels)):
    path = "data/"+str(df['img'][i])
    paths.append(path)


In [7]:
print(len(objects))

23


In [8]:
list_labels = []
for lb in labels:
    if lb in ['Photo caption', 'Photography', 'Font', 'Text', 'Internet meme']:
        pass
    else:
        list_labels.append(lb)
labels.append(list_labels) 

In [9]:
list_objects = []



for i in range(len(objects)):
    for j in range(len(objects[i])):
        if objects[i][j][1]>0:
            
            list_objects.append(objects[i][j][0])
list_objects = set(list_objects)
list_objects = list(list_objects)

In [11]:
my_df = pd.DataFrame({'path': paths,'text': text[:23], 'labels': labels[:23], 'objects':list_objects[:23]
                      })

In [12]:
new_df = df.merge(my_df, left_on='img', right_on='path')
new_df = new_df.rename(columns={'text_x':'text', 'text_y':'ext_text'})
new_df.drop('path', axis=1, inplace=True)
new_df['objects'] = new_df['objects'].apply(set).apply(list)

In [13]:
import nltk
from nltk.corpus import wordnet
import re
nltk.download('wordnet')

def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sohamjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
df['no_elong_text'] = df['text'].apply(lambda x: replaceElongated(x))
new_df['no_elong_text'] = new_df['text'].apply(lambda x: replaceElongated(x))

In [15]:
from functools import partial

with open('slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True)
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])

In [16]:
df['no_slang'] = df['no_elong_text'].apply(lambda x: replaceSlang(x))
new_df['no_slang'] = new_df['no_elong_text'].apply(lambda x: replaceSlang(x))

In [17]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, classification_report, plot_confusion_matrix, plot_roc_curve
nltk.download('stopwords')


new_df = new_df
X = new_df[['no_slang']]

y = np.array(new_df['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tfid1 = TfidfVectorizer(stop_words=stopwords.words('english'), min_df=25)
X1 = tfid1.fit_transform(X_train['no_slang'])
X11 = tfid1.transform(X_test['no_slang'])
X_text_test = pd.DataFrame(X11.toarray())
X_text = pd.DataFrame(X1.toarray())



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sohamjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [63]:
X_train = X_train.to_numpy()



AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
import pickle


In [ ]:
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
pickle.dump(model, open("logistic.pkl", 'wb'))

In [ ]:
from sklearn.naive_bayes import BernoulliNB 

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
pickle.dump(model,open("nai.pkl",'wb'))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
pickle.dump(model, open("ada.pkl", 'wb'))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf=RandomForestClassifier(min_samples_leaf=10, random_state=42)
clf.fit(X_train, y_train)
pickle.dump(model, open("rfc.pkl", 'wb'))